In [1]:
VER = 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd, numpy as np
import pickle, glob, gc

from collections import Counter
import itertools

# multiprocessing 
import psutil
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N Cores : {N_CORES}")
from multiprocessing import Pool

N Cores : 8


In [4]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_test(files):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(files)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})
train = load_test('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-validation/*_parquet/*')
print('train data has shape',train.shape)

train data has shape (171638757, 4)


In [6]:
def statistics_process(df):
    df = df.groupby('aid').agg({'aid':'count',
                                'session':['nunique','count'],
                                'ts':['min','max','sum','mean','std'],
                                'type':['min','max','sum','mean','std']})
    df.columns = ['item_item_count',
                  'item_user_nunique','item_user_count',
                  'item_ts_min','item_ts_max','item_ts_sum','item_ts_mean','item_ts_std',
                  'item_type_min','item_type_max','item_type_sum','item_type_mean','item_type_std']
    return df# sum,mean,min,max,count,std,nunique,cumcount

In [10]:
train = statistics_process(train)
train.to_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/item/val_item{VER}.pqt')

In [11]:
train.head()

,item_item_count,item_user_nunique,item_user_count,item_ts_min,item_ts_max,item_ts_sum,item_ts_mean,item_ts_std,item_type_min,item_type_max,item_type_sum,item_type_mean,item_type_std
aid,,,,,,,,,,,,,
0,38,30,38,1659345217,1661709325,6.309220e+10,1.660321e+09,696458.677960,0,0,0.0,0.000000,0.000000
1,32,29,32,1659719430,1660769860,5.313174e+10,1.660367e+09,244013.424859,0,1,1.0,0.031250,0.176777
2,16,15,16,1659342437,1661604076,2.657137e+10,1.660711e+09,525934.888782,0,0,0.0,0.000000,0.000000
3,1525,852,1525,1659352803,1661722418,2.532962e+12,1.660959e+09,367484.354786,0,2,127.0,0.083279,0.314171
4,148,103,148,1659363904,1661679204,2.457326e+11,1.660355e+09,633532.140767,0,1,5.0,0.033784,0.181286


In [12]:
# FREE MEMORY
del train
_ = gc.collect()

In [13]:
test = load_test('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/original/*_parquet/*')
print('Test data has shape',test.shape)

Test data has shape (223644219, 4)


In [14]:
test = statistics_process(test)
test.to_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/item/test_item{VER}.pqt')